In [1]:
import pymarc as pm
import pandas as pd

#### Get author + title from b4u 

In [2]:
def fix_name(name):
    name = name.strip(' ,')
    spl = name.split(',')
    return f'{spl[1]} {spl[0]}' if len(spl) == 2 else name

def add_ill_aut( function, authority, author, illustrator): 
    if function == 'ill':
        illustrator += ';' + authority
    if function == 'aut':
        author += ';' + authority  
    return illustrator, author    

In [10]:
b4u = {'Book Name':[], 'Author NK':[], 'Illustrator NK': [], 'Pages': [], 'Publication Year':[], 'Original Language': [], 'Text Language': [],  'B4U NK':[]}
# Little Red Riding Hood -> Little Red Riding Hood (FT Shape Books)
# Seasons -> Seasons (Learning Wheel)
with open('b4u.mrc', 'rb') as f:
   reader = pm.MARCReader(f)
   for record in reader:
        print(record)
        # Add names to these variables
         
        illustrator = ''
        author = ''
        year = None
        title = None
        language_orig = 'cze'
        language_text = 'cze'

        # Get Language from 041
        for field in record.get_fields('041'):
            for subfield in field.get_subfields('a'):   
                language_text = subfield  
            for subfield in field.get_subfields('h'):   
                language_orig = subfield
              
        
        # Get Author from 100
        for field in record.get_fields('100'):
            subfields = field.subfields_as_dict()
            authority = fix_name(subfields['a'][0])
            for function in subfields['4']:
                illustrator, author  = add_ill_aut(function, authority, author, illustrator)  
        
        # Get Title    
        for field in record.get_fields('245'):
            for subfield in field.get_subfields('a'):   
                title = subfield.strip(' ,:/') 
                title = 'Seasons (Learning Wheel)' if title == 'Seasons' else title 
                title = 'Little Red Riding Hood (FT Shape Books)' if title == 'Little Red Riding Hood' else title  
                break
        
        #Get Year
        for field in record.get_fields('264'):
            for subfield in field.get_subfields('c'):   
                year = int(''.join(i for i in subfield if i.isdigit()))      
        
        # Get Pages
        for field in record.get_fields('300'):
            subfields = field.subfields_as_dict()
            if 'a' in subfields.keys():
                pages = int(''.join(i for i in subfields['a'][0] if i.isdigit()))     
        
        # Get info from 700   
        for field in record.get_fields('700'):
            subfields = field.subfields_as_dict()
            if '4' in subfields.keys() and len(subfields['a']) == len(subfields['4']):  
                for authority, function in zip(subfields['a'], subfields['4'] ):
                    authority = fix_name(authority)
                    illustrator, author  = add_ill_aut(function, authority, author, illustrator)  
        
        b4u['Illustrator NK'].append(illustrator.strip('; ') if illustrator != '' else None)
        b4u['Author NK'].append(author.strip('; ')) 
        b4u['Book Name'].append(title) 
        b4u['Pages'].append(pages) 
        b4u['Publication Year'].append(year)
        b4u['Original Language'].append(language_orig)
        b4u['Text Language'].append(language_text)
        b4u['B4U NK'].append(True)
print(len(b4u['Book Name']))            
df_b4u = pd.DataFrame(b4u) 
df_b4u = df_b4u.drop_duplicates()
df_b4u.to_excel('excel tables/B4U Marc.xlsx')     

=LDR  02034nam a2200469 a 4500
=001  nkc20112180349
=003  CZ\PrSKC
=005  20110715134304.0
=007  ta
=008  110401s2011\\\\xr\ab\\c\\\\\\000\0\cze
=015  \\$acnb002180349
=020  \\$a978-80-204-2388-7$q(váz.)
=035  \\$a(OCoLC)730408710
=040  \\$aBOA001$bcze$dABA001
=072  \7$a671$xPředměty z drahých kovů$2Konspekt$919
=072  \7$a0/9-053.2$xLiteratura pro děti a mládež (naučná)$2Konspekt$926
=080  \\$a671$2MRF
=080  \\$a379.824.5$2MRF
=080  \\$a(0.053.2)$2MRF
=080  \\$a(084.11)$2MRF
=100  1\$aTůma, Tomáš,$d1974-$7mzk2003203302$4aut$4ill
=245  10$aPoklady světa :$bpříručka hledače pokladů /$c[Tomáš Tůma, Oldřich Růžička ; ilustrace Tomáš Tůma]
=246  30$aPříručka hledače pokladů
=250  \\$a1. vyd.
=260  \\$aPraha :$bMladá fronta,$c2011
=300  \\$a53 s. :$bbarev. il., mapy ;$c28 cm
=500  \\$aObsahuje 6 příl. vlepených do knihy
=500  \\$aPro Mladou frontu vytvořilo B4U Publishing
=520  2\$aKniha seznamuje s legendami a informacemi o největších dosud nenalezených pokladech světa (poklady templářských 

In [ ]:
df_b4u = pd.read_excel('excel tables/B4U Marc.xlsx') 
podklady_albatros = pd.read_excel('excel tables/Přehled Albatros a Scraping.xlsx')
podklady_albatros.Scraping = podklady_albatros.Scraping.apply(lambda x: True if x == 1 else None)
podklady_albatros.Albatros = podklady_albatros.Albatros.apply(lambda x: True if x == 1 else None) 
df = pd.merge(podklady_albatros, df_b4u, how="outer", on=["Book Name", "Book Name"])
df = df.drop(['Unnamed: 0'], axis=1)
df.to_excel('excel tables/Přehled vše.xlsx')